Imprimo la hubicacion de la variable de entorno JAVA_HOME

Debe ser JAVA 64bits y la ruta no debe tener espacios ( Windows )

In [2]:
echo %JAVA_HOME%

C:\Progra~1\Java\jre1.8.0_271


Se configura pyspark para utilizar 20GB de RAM

In [3]:
import os
import pyspark

memory = '20g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

sc = pyspark.SparkContext('local[*]')

In [4]:
from pyspark.sql.types import StructType, DateType, IntegerType, StringType, DoubleType, BooleanType
from pyspark.sql.functions import datediff

In [5]:
sqlContext = pyspark.SQLContext(sc)

Date – The date of the file in yyyy-mm-dd format.

Serial Number – The manufacturer-assigned serial number of the drive.

Model – The manufacturer-assigned model number of the drive.

Capacity – The drive capacity in bytes.

Failure – Contains a “0” if the drive is OK. Contains a “1” if this is the last day the drive was operational before failing.


In [6]:
schema = StructType() \
      .add("Date", DateType(),True) \
      .add("Serial_Number",StringType(),True) \
      .add("Model",StringType(),True) \
      .add("capacity_bytes",DoubleType(),True) \
      .add("Failure",IntegerType(),True)

In [7]:
dia = sqlContext.read.options(header='True', delimiter=',') \
        .schema(schema) \
        .csv("../bigdata/**")

In [8]:
dia.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Serial_Number: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- capacity_bytes: double (nullable = true)
 |-- Failure: integer (nullable = true)



In [9]:
dia.registerTempTable("diskData")

In [10]:
sqlContext.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        | diskdata|       true|
+--------+---------+-----------+



Total de discos Discos del Datacenter con su fecha de Puesta en funcionamiento y dia de retiro

In [25]:
df_AllDisksDayInDayOut = sqlContext.sql('Select Serial_Number, Model, Min(Date) as DayIn, Max(Date) as DayOut, Count(1) as Count from diskData group by Serial_Number, Model order by Model')

In [26]:
df_AllDisksDayInDayOut.createOrReplaceTempView('tbl_AllDisksDayInDayOut')

Se agrega nueva tabla tbl_AllDisksDayInDayOut

In [27]:
sqlContext.sql('show tables').show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|        |            diskdata|       true|
|        |tbl_alldisksdayin...|       true|
+--------+--------------------+-----------+



Esquema de tabla tbl_AllDisksDayInDayOut

In [28]:
df_AllDisksDayInDayOut.printSchema()

root
 |-- Serial_Number: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- DayIn: date (nullable = true)
 |-- DayOut: date (nullable = true)
 |-- Count: long (nullable = false)



Cantidad de discos por modelo que existen.

In [ ]:
sqlContext.sql('Select Model, Count(1) as Count from tbl_AllDisksDayInDayOut group by Model order by Count desc').show()

Listar la cantidad promedio de días que un dura un disco duro en operacion según su modelo.

In [ ]:
sqlContext.sql('Select CAST(AVG(DATEDIFF(DayOut, DayIn)) AS DECIMAL(10,2)) as Days, Model from tbl_AllDisksDayInDayOut Group by Model order by Days desc').show()

In [ ]:
sqlContext.sql('Select Count(1) as Cuenta, Model from diskData group by Model, Serial_Number order by Cuenta desc').show()

In [ ]:
sqlContext.sql('Select Count(1) as Cuenta, Model from diskData group by Model, Serial_Number order by Cuenta desc')\
          .repartition(1)\
          .write.format("parquet")\
          .save("output/parquet/transacciones", mode="OVERWRITE")

In [ ]:
df_averagedia.createOrReplaceTempView('tblAverageDia')

Cantidad de TB en el Datacenter por modelo de disco duro

In [21]:
sqlContext.sql('select Model, CAST(SUM(capacity_bytes/1099511627776) AS DECIMAL(10,2)) as TB from diskData group by model order by TB desc').show()

+--------------------+-----------+
|               Model|         TB|
+--------------------+-----------+
|HGST HMS5C4040BLE640|83827074.69|
|HGST HUH721212ALN604|64281208.75|
|       ST12000NM0008|42626846.54|
| TOSHIBA MG07ACA14TA|37967657.99|
|HGST HMS5C4040ALE640|37739576.47|
|Hitachi HDS5C3030...|18124970.05|
|         ST6000DX000|17138601.81|
|Hitachi HDS5C4040...|16017770.17|
|       ST10000NM0086|12121416.06|
|HGST HUH721212ALE600| 9911802.31|
|Hitachi HDS722020...| 9654412.63|
|HGST HUH728080ALE600| 7785900.79|
|       ST12000NM001G| 7482419.31|
|         ST3000DM001| 6017900.54|
|Hitachi HDS723030...| 3901592.00|
|        WDC WD60EFRX| 3743580.24|
|        WDC WD30EFRX| 3470687.38|
|        ST31500541AS| 1972022.64|
|         ST4000DX000| 1068175.19|
| TOSHIBA MD04ABA400V|  951620.52|
+--------------------+-----------+
only showing top 20 rows



Top 5 modeos de discos más confiables por año

In [ ]:
sqlContext.sql('').show()

Top 5 modeos de discos más confiables por año

In [ ]:
sqlContext.sql('').show(150)

In [ ]:
Cantidad de discos que fallaron por año vs discos en el data center

In [24]:
sqlContext.sql('select Count(Failure) as Failures, Year(Date) as Year from diskData where Failure = 1 group by year(Date) order by Year').show()

+--------+----+
|Failures|Year|
+--------+----+
|     740|2013|
|    2206|2014|
|    1429|2015|
|    1431|2016|
|    1556|2017|
|    1381|2018|
|    2263|2019|
|    1093|2020|
+--------+----+



In [29]:
sqlContext.sql('select count(distinct Serial_Number) as Count, Year(Date) as Year from diskData group by Year(date)').show()

+-----------------------------+----+
|count(DISTINCT Serial_Number)|Year|
+-----------------------------+----+
|                       124326|2018|
|                        62898|2015|
|                        29072|2013|
|                        47793|2014|
|                       136568|2019|
|                       162997|2020|
|                        81173|2016|
|                       108282|2017|
+-----------------------------+----+



In [35]:
sqlContext.sql('Select Count, Failures, Failures.Year from (select count(distinct Serial_Number) as Count, Year(Date) as Year from diskData group by Year(date)) as Disks inner Join (select Count(Failure) as Failures, Year(Date) as Year from diskData where Failure = 1 group by year(Date) order by Year) as Failures on Disks.Year = Failures.Year order by Failures.Year').show()

+------+--------+----+
| Count|Failures|Year|
+------+--------+----+
| 29072|     740|2013|
| 47793|    2206|2014|
| 62898|    1429|2015|
| 81173|    1431|2016|
|108282|    1556|2017|
|124326|    1381|2018|
|136568|    2263|2019|
|162997|    1093|2020|
+------+--------+----+



In [ ]:
selectedData = dia.select("Date", "Serial_Number")

In [ ]:
selectedData.write.csv('test2.csv')